In [1]:
import sys
#sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
from metatlas import metatlas_objects as metob
import qgrid
%matplotlib notebook
import pandas as pd
print metob.database
from metatlas import  gui
from ipywidgets import widgets
from IPython.display import display


<Database(mysql+pymysql://localhost/metatlas)>


In [4]:
#c = metob.retrieve('Groups', description='15_42h')

atlases = widgets.Select(description = 'Atlases:', options = [])
compounds = widgets.Select(description = 'Compounds:', options = [])
search_string = widgets.Text(description = "Atlas Name")
compound_string = widgets.Text(description = "Compound Name")
display(search_string)
display(compound_string)
display(compounds)
display(atlases)


d5-benzoic acid
20151130_LS_Negative_Hilic_QExactive_Archetypes_ISTDs
20151130_LS_Negative_Hilic_QExactive_Archetypes_ISTDs


In [7]:
def new_atlas_list (sender):
    wild_card = search_string.value
    atlas = metob.retrieve('Atlas',name= wild_card, username='*')
    atlases_list = list()
    for cc in atlas:
        atlases_list.append(cc.name)    
           
    atlases.options = atlases_list    
  
                            
                            
                            
def select_atlas(atlas_name):
    atlas = metob.retrieve('Atlas',name= atlas_name, username='*')   
    compounds_list = list()
    for cc in atlas:
        
        for x in cc.compound_identifications:
            z = x.compound
            
            for zz in z:
                compounds_list.append(zz.name)

    
    
    compounds.options = compounds_list                        
                            
    

def new_compounds_list(sender):
    compounds.options = [atlases.value]

def print_compound(value):
    print value['new']
    
def print_atlas(sender):
    print atlases.value
    
search_string.observe(new_atlas_list)
atlases.observe(print_atlas)
print atlases.value
i = widgets.interactive(select_atlas, atlas_name=atlases)
compounds.observe(print_compound, 'value')
print i


None


In [ ]:
atlas = metob.retrieve('Atlas', name='%_LS_%', username='*')

print len(atlas)

atlases = dict()

cc = atlas[1]

for x in cc.compound_identifications:
    print x

#print x.
#print type(x.mz_references)
#print x.mz_references
#print x.compound
#print '-'*100
#print metob.to_dataframe(x.mz_references)
#        x = cc.compound_identifications[1]
#        for xx in x.mz_references:
#            pass
            #print xx
            #print '-'*100
            #break
#        break
#    break
    
atlases = {'name': ['1', '2', '3'],
           'compounds': ['a', 'b', 'c'],
           'mz_references': ['d', 'e', 'f'],
           'rt_refences': ['g', 'h', 'i'],
           'frag_references': ['j', 'k', 'l']
          }
    
pd.DataFrame.from_dict(atlases)
grid = gui.create_qgrid([])
grid.df = pd.DataFrame.from_dict(atlases)

display(grid)

#x = atlas[1]
#x.compound_identifications

In [ ]:
#make file info template sheet by selecting a folder:
# bpb@edison06:/project/projectdirs/metatlas/raw_data/lpsilva> ls -lta
# total 3584
# drwxrws--- 17 silvest metatlas 131072 Oct 11 10:18 ..
# drwxrws---  2 silvest metatlas 131072 Sep 29 15:30 Actinobacillus test
# drwxrws---  2 silvest metatlas 262144 Sep 29 14:13 20150521_LPSilva_Actino_C18_NEG_51isolates
# drwxrws---  2 silvest metatlas 262144 Sep 29 13:43 20150514_LPSilva_Actino_C18_POS_51isolates
# drwxrws---  2 silvest metatlas 262144 Sep 29 13:09 20150512_LPSilva_Actino_HILIC_NEG_51isolates
# drwxrws---  2 silvest metatlas 262144 Sep 29 11:46 20150504_LPSilva_Actino_HILIC_POS_51isolates
# files = metob.retrieve('LcmsRun',mzml_file='%katezh/20151016_%')
files = metob.retrieve('LcmsRun',experiment='20160203_KBL-BC_Root-Exudate_Hilic_QExactive_Trial-Run', username='*')

# files = metob.retrieve('LcmsRun',mzml_file='%_LPSilva_Actino_%_51isolates%', username='*')
flist = []
for f in files:
    flist.append(f.mzml_file)
import numpy as np
flist = np.unique(flist)
print len(flist)


In [ ]:
files[0]

In [ ]:
#dump all the files to a spreadheet, download it, and make a "filled in" one.
with open('FileInfo_Sheet_Unique_20160203_KBL-BC_Root-Exudate_Hilic_QExactive_Trial-Run.tab','w') as fid:
    fid.write('mzml_file\tgroup\tdescription\n')
    for f in flist:
        fid.write('%s\t\t\n'%f)


In [ ]:
#Download the fileinfo sheet and fill it in!
df = pd.read_csv('FileInfo_Sheet_Unique_20160203_KBL-BC_Root-Exudate_Hilic_QExactive_Trial-Run.tab',sep='\t')
df

In [ ]:
grouped = df.groupby(by='group')
# list(grouped)

In [ ]:
grouped.groups.keys()

In [ ]:
# metob.retrieve('LcmsRun',mzml_file='%0150115_pHILIC_NEG_MSMS_R2A-1An%',username='*')

In [ ]:
for g in grouped.groups.keys():
    indices = grouped.groups[g]
    myGroup = metob.Group()
    myGroup.name = '%s'%g
    myGroup.description = df.loc[indices[0],'description']
    file_set = []
    for i in indices:
        file_set.append(metob.retrieve('LcmsRun',mzml_file='%%%s'%df.loc[i,'mzml_file'],username='*')[0])
    myGroup.items = file_set
    metob.store(myGroup)

In [ ]:
g = metob.retrieve('Groups',name = '%_KBL_PS_%')
print len(g)
for gg in g:
    print gg.name
    for i in gg.items:
        print i.name
    print " "